In [1]:
import findspark
from memory_profiler import profile
findspark.init('/home/Ilan/miniconda/envs/garage/lib/python2.7/site-packages/pyspark')
findspark.add_jars('/usr/local/bin/mysql-connector-java-5.1.46-bin.jar')
from pyspark import SparkContext , SQLContext
from pyspark.sql import SparkSession
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers
from Trax.Data.Projects.Connector import ProjectConnector
import pandas as pd


#sc = SparkContext(appName="myAppName")
spark = SparkSession.builder.appName("run_etl").config("spark.driver.memory","4g").config("spark.executor.memory", "4g").config("spark.driver.cores", "4").config("spark.driver.maxResultSize", "4").config("spark.ssl.protocol", "TLSv1.1").getOrCreate()



### project params 

In [2]:
project_name = 'jnjuk'
env = 'prod'
start_date = '2018-07-01'
end_date =  '2018-07-02'
batch_size = 80000

In [7]:
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers

Config.init(app_name=' ', default_env=env ,config_file='~/theGarage/Trax/Apps/Services/KEngine/k-engine-prod.config')
Config.PROD

connector = ProjectConnector(project_name,DbUsers.CalculationEng)

Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: Failed to retrieve credentials from vault: exception is :Login to vault failed 1 tiems. last login resulted errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - Traceback (most recent call last):
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/Credentials.py", line 89, in get_credentials
    token = cls.cloud_adapter.get_vault_token(force)
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 94, in get_vault_token
    self._login_vault_request()
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 70, in _login_vault_request
    self._check_for_

In [111]:
connector.project_params['rds_name']
project_url = 'jdbc:mysql://{}/report'.format(connector.project_params['rds_name'])

results = '''    
                    (SELECT 
                        report.kpi_level_2_results.*
                    FROM
                        report.kpi_level_2_results,
                        probedata.session
                    WHERE
                        probedata.session.pk = report.kpi_level_2_results.session_fk
                            AND probedata.session.visit_date BETWEEN '{}' AND '{}')tmp_kpi_level_2_results '''.format(start_date, end_date)

static = '''(SELECT * FROM  static.kpi_level_2 where static.kpi_level_2.kpi_calculation_stage_fk = 3) static_kpi '''
categories =  '''(select * from static_new.category) categories '''

In [9]:
connector = ProjectConnector('diageous',DbUsers.CalculationEng)
meta_results = '''    
            SELECT 
                count(*) count, 
                MIN(probedata.session.pk) min,
                MAX(probedata.session.pk) max
            FROM
                report.kpi_level_2_results,
                probedata.session
            WHERE
                probedata.session.pk = report.kpi_level_2_results.session_fk
                    AND probedata.session.visit_date BETWEEN '{}' AND '{}';'''.format(start_date, end_date)
kpi_results_meta_data = pd.read_sql_query(meta_results, connector.db)
connector.disconnect_rds()

Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: get request returned errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - get_credentials: Failed to retrieve credentials from vault: exception is :Login to vault failed 1 tiems. last login resulted errors: [u'entry for role ilanp@traxretail.com not found'] (extra=None)
Log was not initialized - Traceback (most recent call last):
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/Credentials.py", line 89, in get_credentials
    token = cls.cloud_adapter.get_vault_token(force)
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 94, in get_vault_token
    self._login_vault_request()
  File "/home/Ilan/dev/theGarage/Trax/Cloud/Services/Connector/ConnectorAdapter.py", line 70, in _login_vault_request
    self._check_for_

In [10]:
lowerBound = kpi_results_meta_data.loc[0]['min']
upperBound = kpi_results_meta_data.loc[0]['max']
count_of_row = kpi_results_meta_data.loc[0]['count']

if (count_of_row / batch_size ) > 1 :
    number_of_partition = (count_of_row / batch_size )
else: 
    number_of_partition = 1 


count of rows : 9227
 number of partition:1


In [13]:
static_kpi = spark.read.jdbc(url=project_url,
                          table=static,                          
                          properties={"user": connector.dbuser.username, 
                                      "password":connector.dbuser.cred,
                                      "driver": 'com.mysql.jdbc.Driver'}).cache()


static_kpi.count()


11

In [14]:
kpi_results = spark.read.jdbc(url=project_url,
                          table=results,                          
                          properties={"user": connector.dbuser.username, 
                                      "password":connector.dbuser.cred,
                                      "partitionColumn":"tmp_kpi_level_2_results.session_fk" ,
                                      "lowerBound":"{}".format(lowerBound),
                                      "upperBound":"{}".format(upperBound),
                                      "numPartitions":"{}".format(number_of_partition),
                                      "driver": 'com.mysql.jdbc.Driver'}).cache()

kpi_results.count()


342696

In [ ]:
categories_df = spark.read.jdbc(url=project_url,
                          table=categories,                          
                          properties={"user": connector.dbuser.username, 
                                      "password":connector.dbuser.cred,
                                      "driver": 'com.mysql.jdbc.Driver'}).cache()


categories_df.count()

In [ ]:
### statistics 

In [51]:
static_kpi.select("client_name").distinct().count()

11

### test 1 - kpi not calculated 

In [21]:
from pyspark.sql.functions import countDistinct

merged_kpi_results = static_kpi.join(kpi_results, static_kpi.pk == kpi_results.kpi_level_2_fk ,how='left')
filtered = merged_kpi_results.filter('result is null')
df = filtered.groupBy("client_name","result").count().withColumnRenamed('count', 'result_count')
df2 = filtered.groupBy("client_name").agg(countDistinct("session_fk").alias("session count"))
df.join(df2, df2.client_name == df.client_name).show()




+------------------+------+------------+------------------+-------------+
|       client_name|result|result_count|       client_name|session count|
+------------------+------+------------+------------------+-------------+
|eye_hand_level_sos|  null|          13|eye_hand_level_sos|            8|
+------------------+------+------------+------------------+-------------+



### test -invalid % results

In [30]:
t = merged_kpi_results.select("client_name").filter('result is null').groupBy("client_name").count().toPandas()
t.to_csv("file3.csv")

In [62]:
import pyspark.sql.functions as F

#merged_kpi_results.select("session_fk","client_name","result").filter('is_percent = 1 and result < 0 or result > 1').show(1000 ,False)
#merged_kpi_results.select("client_name").filter('is_percent = 1 and result < 0 or result > 1').groupBy("client_name").count().show(1000 ,False)

total_sessions= merged_kpi_results.select("session_fk").distinct().count()

filtered = merged_kpi_results.filter('is_percent = 1 and result < 0 or result > 1')
df = filtered.groupBy("client_name").count().withColumnRenamed('count', 'result_count').withColumnRenamed('client_name', 'name')
df2 = filtered.groupBy("client_name").agg(countDistinct("session_fk").alias("session_count"))
test_restuls = df.join(df2, df2.client_name == df.name)

test_restuls.select('client_name','result_count',"session_count", (( F.col('session_count') / total_sessions)* 100).alias("session_count%")).show(1000,False)



+-----------------------------------+------------+-------------+-----------------+
|client_name                        |result_count|session_count|session_count%   |
+-----------------------------------+------------+-------------+-----------------+
|promo                              |9           |9            |37.5             |
|PRODUCT_PRESENCE_IN_CATEGORY_CUSTOM|659         |16           |66.66666666666666|
|store_assortment - SKU             |659         |16           |66.66666666666666|
|eye_hand_level_sos                 |552         |22           |91.66666666666666|
|promo - SKU                        |158         |9            |37.5             |
+-----------------------------------+------------+-------------+-----------------+



### test -results with value 0 


In [18]:
import pyspark.sql.functions as F

In [73]:
results_with_zero2 = merged_kpi_results.select("session_fk","client_name","result").filter( 'result == 0')


merged_kpi_results2 = merged_kpi_results.groupBy('client_name').count().withColumnRenamed('count', 'total_count')
results_with_zero2 = results_with_zero2.groupBy('client_name').count().withColumnRenamed('count', 'total_zero_count').withColumnRenamed('client_name', 'client_name_zero')
merged_test = merged_kpi_results2.join(results_with_zero2 , results_with_zero2.client_name_zero == merged_kpi_results2.client_name, how= 'left' )

print type (merged_test)
res= merged_test.select('client_name',( F.col('total_zero_count') / F.col('total_count'))* 100 ) .filter('total_zero_count is not null' )

res.show(100, False)
#todo write .all to one file 
# sults_with_zero.write.csv('/tmp/results_with_zero.csv')

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------------------------------------------+----------------------------------------+
|client_name                                    |((total_zero_count / total_count) * 100)|
+-----------------------------------------------+----------------------------------------+
|promo                                          |10.0                                    |
|secondary placement location visibility quality|100.0                                   |
|PRODUCT_PRESENCE_IN_CATEGORY_CUSTOM            |23.815028901734102                      |
|store_assortment - SKU                         |23.815028901734102                      |
|eye_hand_level_sos                             |40.526315789473685                      |
|secondary placement location quality           |91.66666666666666                       |
|DST_BY_CATEGORY                                |2.9850746268656714                      |
|promo - SKU                                    

In [91]:
total_sessions= merged_kpi_results.select("session_fk").distinct().count()


filtered = merged_kpi_results.filter( 'result == 0')
df = filtered.groupBy('client_name').count().withColumnRenamed('count', 'results_zero_count').withColumnRenamed('client_name', 'name')
df2 = filtered.groupBy("client_name").agg(F.countDistinct("session_fk").alias("session_count"))
df3 = merged_kpi_results.groupBy('client_name').count().withColumnRenamed('count', 'total_count') \
.withColumnRenamed('client_name', 'name2')
test_results2 = df.join(df2, df2.client_name == df.name).join(df3, df3.name2 == df.name)
#test_results2 = df.join(test_results, test_results.client_name == df.name)
test_results2.select('client_name', \
                     'results_zero_count', \
                     # 'total_count',\
                     # "session_count",\
                     ((F.col('results_zero_count') / F.col('total_count')) * 100).alias("results%(out of all results)") ,\
                     ((F.col('session_count') / total_sessions) * 100).alias("session_count%(out of all sessions)") \
                     
                    ).show(1000, False)


+-----------------------------------------------+------------------+----------------------------+-----------------------------------+
|client_name                                    |results_zero_count|results%(out of all results)|session_count%(out of all sessions)|
+-----------------------------------------------+------------------+----------------------------+-----------------------------------+
|promo                                          |1                 |10.0                        |4.166666666666666                  |
|secondary placement location visibility quality|24                |100.0                       |100.0                              |
|PRODUCT_PRESENCE_IN_CATEGORY_CUSTOM            |206               |23.815028901734102          |58.333333333333336                 |
|store_assortment - SKU                         |206               |23.815028901734102          |58.333333333333336                 |
|eye_hand_level_sos                             |385          

In [92]:
### test - uniqe resutls per kpi (there is only 2 result type for kpi)

test_results = merged_kpi_results.groupBy('client_name','result').count().withColumnRenamed('count', 'total_zero_count').withColumnRenamed('client_name', 'client_name_zero')
validation = merged_kpi_results2.join(test_results , test_results.client_name_zero == merged_kpi_results2.client_name, how= 'left' )

validation.select('client_name' ,'result' , (F.col('total_zero_count') / F.col('total_count'))).filter((F.col('total_zero_count') / F.col('total_count') ) > 0.5  ).show()


+--------------------+------+--------------------------------+
|         client_name|result|(total_zero_count / total_count)|
+--------------------+------+--------------------------------+
|secondary placeme...|   0.0|                             1.0|
|PRODUCT_PRESENCE_...| 100.0|               0.761849710982659|
|store_assortment ...| 100.0|               0.761849710982659|
|  eye_hand_level_sos| 100.0|              0.5452631578947369|
|secondary placeme...|   0.0|              0.9166666666666666|
|         promo - SKU|   0.0|              0.6425339366515838|
+--------------------+------+--------------------------------+



In [97]:
#### std per kpi 
merged_kpi_results.groupBy('client_name').agg({"result":"stddev"}).show()

+--------------------+-------------------+
|         client_name|     stddev(result)|
+--------------------+-------------------+
|               promo|  15.98914958179105|
|secondary placeme...|                0.0|
|PRODUCT_PRESENCE_...|  42.61979853981476|
|store_level_assor...| 0.1267231540019259|
|store_assortment ...|  42.61979853981476|
|  eye_hand_level_sos|  48.78212290333069|
|secondary placeme...|0.28232985128663995|
|     DST_BY_CATEGORY| 0.2387539758185753|
|         promo - SKU|  47.97967303700703|
|     OOS_BY_CATEGORY|0.23875398002180076|
|OOS_BY_DYNAMIC_AS...|0.12672312700691943|
+--------------------+-------------------+



In [110]:
merged_kpi_results.filter('denominator_type_fk = 4').count()

999